In [ ]:

!git clone https://github.com/ggerganov/llama.cpp


Cloning into 'llama.cpp'...
remote: Enumerating objects: 34249, done.
remote: Counting objects: 100% (7642/7642), done.
remote: Compressing objects: 100% (639/639), done.
remote: Total 34249 (delta 7366), reused 7055 (delta 6996), pack-reused 26607 (from 1)
Receiving objects: 100% (34249/34249), 57.85 MiB | 13.45 MiB/s, done.
Resolving deltas: 100% (24834/24834), done.


In [ ]:
!cd llama.cpp && GGML_CUDA=1 make && pip install -r requirements/requirements-convert-hf-to-gguf.txt

I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG -DGGML_USE_OPENMP -DGGML_USE_LLAMAFILE -DGGML_USE_CUDA -I/usr/local/cuda/include -I/usr/local/cuda/targets/x86_64-linux/include -DGGML_CUDA_USE_GRAPHS  -std=c11   -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -fopenmp -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -g -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread -fopenmp  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -Iggml/include -Iggml/src -Iinclude -Isrc -Icommon -D_XOPEN_SOURCE=600 -D_GNU_S

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
model_name = "armin06/Llama-2-7b-chat-finetune_45k"

In [ ]:
methods = ['q4_k_m']

In [ ]:
base_model = "./original_model/"
quantized_path = "./quantized_model/"


In [ ]:

snapshot_download(repo_id=model_name, local_dir=base_model , local_dir_use_symlinks=False)
original_model = quantized_path+'/FP16.gguf'


Fetching 11 files:   0%|          | 0/11 [00:00<?, ?it/s]

In [ ]:
!mkdir ./quantized_model/

In [ ]:

!python /content/llama.cpp/convert_hf_to_gguf.py ./original_model/ --outtype f16 --outfile ./quantized_model/FP16.gguf

In [ ]:

import os

In [ ]:
!apt-get update
!apt-get install -y cuda

In [ ]:

for m in methods:
    qtype = f"{quantized_path}/{m.upper()}.gguf"
    os.system("./llama.cpp/llama-quantize "+quantized_path+"/FP16.gguf "+qtype+" "+m)

In [ ]:
! /content/llama.cpp/main -m ./quantized_model/Q4_K_M.gguf -n 90 --repeat_penalty 1.0 --color -i -r "User:" -f llama.cpp/prompts/chat-with-bob.txt



 Please use 'llama-cli' instead.
 See https://github.com/ggerganov/llama.cpp/tree/master/examples/deprecation-warning/README.md for more information.



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from huggingface_hub import HfApi, HfFolder, create_repo, upload_file

In [ ]:

model_path = "./quantized_model/Q4_K_M.gguf" # Your model's local path
repo_name = "qwen-7b-llm"  # Desired HF Hub repository name
repo_url = create_repo(repo_name, private=False)

In [ ]:

api = HfApi()
api.upload_file(
    path_or_fileobj="./quantized_model/Q4_K_M.gguf",
    path_in_repo="Q4_K_M.gguf",
    repo_id="armin06/qwen-7b-llm",
    repo_type="model",
)

Q4_K_M.gguf:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/armin06/qwen-7b-llm/commit/cf90dcc76a91035d1a9fbce24e236e6b94a440a9', commit_message='Upload Q4_K_M.gguf with huggingface_hub', commit_description='', oid='cf90dcc76a91035d1a9fbce24e236e6b94a440a9', pr_url=None, pr_revision=None, pr_num=None)